In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
from scipy.integrate import odeint
import scipy.constants as cons
import scipy.linalg as la
import scipy.optimize as op
from qutip import *

from sympy import *
init_printing(use_unicode=True) # pretty printing

## Lambda Configuration Steady State Solution
$$\displaystyle \left[\begin{matrix}\frac{i \Omega_{p} \left(\rho_{13} - \rho_{31}\right)}{2} + \gamma_{1} \rho_{33} & \frac{i \left(\Omega_{c} \rho_{13} - \Omega_{p} \rho_{32} + 2 \rho_{12} \left(\Delta_{c} - \Delta_{p}\right)\right)}{2} & - \frac{\rho_{13} \left(\gamma_{1} + \gamma_{2}\right)}{2} - \frac{i \left(2 \Delta_{p} \rho_{13} - \Omega_{c} \rho_{12} - \Omega_{p} \rho_{11} + \Omega_{p} \rho_{33}\right)}{2}\\\frac{i \left(- \Omega_{c} \rho_{31} + \Omega_{p} \rho_{23} - 2 \rho_{21} \left(\Delta_{c} - \Delta_{p}\right)\right)}{2} & \frac{i \Omega_{c} \left(\rho_{23} - \rho_{32}\right)}{2} + \gamma_{2} \rho_{33} & - i \Delta_{c} \rho_{23} + \frac{i \Omega_{c} \rho_{22}}{2} - \frac{i \Omega_{c} \rho_{33}}{2} + \frac{i \Omega_{p} \rho_{21}}{2} - \frac{\gamma_{1} \rho_{23}}{2} - \frac{\gamma_{2} \rho_{23}}{2}\\- \frac{\rho_{31} \left(\gamma_{1} + \gamma_{2}\right)}{2} + \frac{i \left(2 \Delta_{p} \rho_{31} - \Omega_{c} \rho_{21} - \Omega_{p} \rho_{11} + \Omega_{p} \rho_{33}\right)}{2} & i \Delta_{c} \rho_{32} - \frac{i \Omega_{c} \rho_{22}}{2} + \frac{i \Omega_{c} \rho_{33}}{2} - \frac{i \Omega_{p} \rho_{12}}{2} - \frac{\gamma_{1} \rho_{32}}{2} - \frac{\gamma_{2} \rho_{32}}{2} & - \rho_{33} \left(\gamma_{1} + \gamma_{2}\right) - \frac{i \left(\Omega_{c} \rho_{23} - \Omega_{c} \rho_{32} + \Omega_{p} \rho_{13} - \Omega_{p} \rho_{31}\right)}{2}\end{matrix}\right]$$

In [ ]:
# -- Three Level System - Lambda Configuration

# Symbol definitions for density matrix components
rho11, rho12, rho13, rho21, rho22, rho23, rho31, rho32, rho33 = symbols('rho11 rho12 rho13 rho21 rho22 rho23 rho31 rho32 rho33')
delta_p, delta_c = symbols('Delta_p Delta_c')

# Symbol definitions for Hamiltonian components
Omega_p, Omega_c = symbols('Omega_p Omega_c')
hbar = symbols('hbar')

# Symbol definitions for Lindbladian components
gamma1, gamma2 = symbols('gamma_1 gamma_2')

p = Matrix([[rho11, rho12, rho13], [rho21, rho22, rho23], [rho31, rho32, rho33]]) # Density Matrix

H_AL = hbar * Matrix([[0, 0, Omega_p/2], [0, -(delta_p - delta_c), Omega_c/2], [Omega_p/2, Omega_c/2, -delta_p]])

# Atomic Transition Operators
p11 = Matrix([[1, 0, 0], [0, 0, 0], [0, 0, 0]])
p22 = Matrix([[0, 0, 0], [0, 1, 0], [0, 0, 0]])
p33 = Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 1]])
p12 = Matrix([[0, 1, 0], [0, 0, 0], [0, 0, 0]])
p21 = Matrix([[0, 0, 0], [1, 0, 0], [0, 0, 0]])
p13 = Matrix([[0, 0, 1], [0, 0, 0], [0, 0, 0]])
p31 = Matrix([[0, 0, 0], [0, 0, 0], [1, 0, 0]])
p23 = Matrix([[0, 0, 0], [0, 0, 1], [0, 0, 0]])
p32 = Matrix([[0, 0, 0], [0, 0, 0], [0, 1, 0]])

# Lidbladian Components
L31 = gamma1 * ((p13 * p * p31) - (p33 * p)/2 - (p * p33)/2)
L32 = gamma2 * ((p23 * p * p32) - (p33 * p)/2 - (p * p33)/2)

# Lindbladian
L = L31 + L32

# Lindbladian Master Equation
liouville = -(I/hbar) * ((H_AL * p) - (p * H_AL))
obe = liouville + L

cond = obe[0] + obe[4] + obe[8]

if simplify(cond) == 0:
    print("The condition p11 + p22 + p33 = 0 holds")
else:
    print("Conndition does not hold")

#simplify(obe)
simplify(H_AL)

In [ ]:
# -- Three Level System - Lambda Configuration Steady State Solution

# Rabi frequencies for probe and coupling fields
Omega_p = 2 * np.pi * 1
Omega_c = 2 * np.pi * 3

wp = (Omega_p*1j)/2
wc = (Omega_c*1j)/2

# Coupling field detuning
dc = 0

# Radiative decay rates
gamma1 = 2 * np.pi * 1
gamma2 = 2 * np.pi * 0

gmean = (gamma1 + gamma2) / 2

# Probe detuning range
dp_range = np.arange(-50,51,0.01)    

sol = np.empty((9, len(dp_range)), dtype='complex')
for dp in range(len(dp_range)):
    
    drho = np.array([[0, 0, wp, 0, 0, 0, -wp, 0, gamma1],                       # p11
                     [0, -(dp_range[dp] - dc)*1j, wc, 0, 0, 0, 0, -wp, 0],      # p12
                     [wp, wc, -dp_range[dp]*1j - gmean, 0, 0, 0, 0, 0, -wp],    # p13
                     [0, 0, 0, (dp_range[dp] - dc)*1j, 0, wp, -wc, 0, 0],       # p21
                     [0, 0, 0, 0, 0, wc, 0, -wc, gamma2],                       # p22
                     [0, 0, 0, wp, wc, -dc*1j - gmean, 0, 0, -wc],              # p23
                     [-wp, 0, 0, -wc, 0, 0, (dp_range[dp]*1j) - gmean, 0, wp],  # p31
                     [0, -wp, 0, 0, -wc, 0, 0, (dc*1j) - gmean, wc],            # p32
                     [0, 0, -wp, 0, 0, -wc, wp, wc, -(gamma1 + gamma2)],        # p33
                     [1, 0, 0, 0, 1, 0, 0, 0, 1]])

    b = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

    rho = la.lstsq(drho, b)[0]
    sol[:, dp] = rho

pop = sol[0] + sol[4] + sol[8]  # total populations

# ---------- Plotting ----------
plt.style.use(['science', 'notebook', 'grid'])

# Populations
plt.plot(dp_range/(2*np.pi), sol[0, :].real, label=r"$|1\rangle$")
plt.plot(dp_range/(2*np.pi), sol[4, :].real, label=r"$|2\rangle$")
plt.plot(dp_range/(2*np.pi), sol[8, :].real, label=r"$|3\rangle$")

# Total population
#plt.plot(dp_range/(2*np.pi), pop.real, label="Total populations")

plt.xlabel('$\Delta_p/2\pi$')
plt.ylabel(r'Population ($\rho$)')

plt.legend(loc='upper right')
#plt.legend(loc='upper right', bbox_to_anchor=(1.48, 0.75))
plt.savefig("Figures/3LS_pop.pdf")

In [ ]:
# Rho13 Im and Re parts
plt.plot(dp_range/(2*np.pi), sol[2, :].imag, label=r'$\rho_{13}$ Im')
plt.plot(dp_range/(2*np.pi), sol[2, :].real, label=r'$\rho_{13}$ Re')
plt.xlabel("$\Delta_p/2\pi$")
plt.ylabel('Population')
plt.legend(loc='upper right')
plt.savefig("Figures/3LS_coherences.pdf")